In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-reviews-2018-full-dataset/amazon_reviews.csv


In [58]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Ensure stopwords are downloaded
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("punkt")
stop_words = set(stopwords.words("english"))

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -3] Temporary failure in name resolution>
[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [59]:
df = pd.read_csv('/kaggle/input/amazon-reviews-2018-full-dataset/amazon_reviews.csv',)

In [60]:
df['rating'].value_counts()

rating
5.0    379783
4.0     73303
3.0     40639
1.0     34021
2.0     23413
Name: count, dtype: int64

In [61]:
df.head(10)

,userName,verified,itemName,description,image,brand,feature,category,price,rating,reviewTime,summary,reviewText,vote
0,Amazon Customer,True,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...","[""Made from deliciously decadent ingredients, ...",['https://images-na.ssl-images-amazon.com/imag...,Toblerone,[],Prime_Pantry,$1.63,5.0,2018-01-01,Five Stars,super smooth and yummy with crunchy bits,0
1,Carla,True,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded i...",['https://images-na.ssl-images-amazon.com/imag...,Prince Of Peace,[],Grocery_and_Gourmet_Food,$6.40,5.0,2018-01-01,Five Stars,Perfect for kombucha,0
2,Jaclyn,True,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",['https://images-na.ssl-images-amazon.com/imag...,Pet Champion,"['Features Bright Pink Lemonade pattern', 'Dua...",Pet_Supplies,$7.99,5.0,2018-01-01,she has an odd shape chest and her pull over h...,Finally a harness that fits my puppy. I really...,0
3,Christinne M,False,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,['https://images-na.ssl-images-amazon.com/imag...,KOH-I-NOOR,"['Rich, pigmented, solid color encased in lacq...",Arts_Crafts_and_Sewing,$14.18,5.0,2018-01-01,Loving!!,I LOVE THEM!! I bought them at Micheals our of...,0
4,Sandy,True,"Parker Urban Premium Ebony Metal Chiseled, Bal...","[""Stunning craftsmanship based on a classic st...",['https://images-na.ssl-images-amazon.com/imag...,Parker,['Ebony black with metallic highlights and a g...,Office_Products,NaN,5.0,2018-01-01,I love this pen,"I love this pen! I love the shape of it, the f...",0
5,Karli S.,True,Merrick Whole Earth Farms Adult Recipe Dry Dog...,['Merrick Whole Earth Farms is a nutritious do...,['https://images-na.ssl-images-amazon.com/imag...,Merrick,['Natural ingredients with added vitamins and ...,Pet_Supplies,NaN,2.0,2018-01-01,No Bueno.,We accidentally purchased this instead of our ...,0
6,patrick,True,Blue Buffalo Life Protection Formula Natural P...,['Because puppyhood is such an important stage...,['https://images-na.ssl-images-amazon.com/imag...,Blue Buffalo,['HIGH QUALITY PUPPY FOOD: Blue Buffalo always...,Pet_Supplies,$28.99,5.0,2018-01-01,... and has had no problems with this food - B...,My 1 year old husky loves it - healthy and has...,0
7,Craig S.,True,Squishy Face Studio Flirt Pole V2 with Braided...,"[""Want An Easy Way To Keep Your Dog Happy And ...",['https://images-na.ssl-images-amazon.com/imag...,Squishy Face Studio,['New and improved V2 developed using customer...,Pet_Supplies,$31.38,3.0,2018-01-01,I like the product,"I like the product, but am sorry that I purcha...",0
8,Super Sanne,True,Penn Plax Net Breeder for Aquarium,['The Net Breeder by Penn Plax is the safe way...,['https://images-na.ssl-images-amazon.com/imag...,Penn Plax,['ISOLATE THE FRY: the Net Breeder Deluxe is t...,Pet_Supplies,$5.49,2.0,2018-01-01,Not worth the money,Very flimsy and cheaply made. We bought 2 and ...,0
9,Matthew Daviskavitch,True,PetSafe Freedom Aluminum Patio Panel Sliding G...,"[""The Pet Safe Freedom Patio Panel Pet Door al...",['https://images-na.ssl-images-amazon.com/imag...,PetSafe,['PERFECT FOR RENTALS: Great for apartments or...,Pet_Supplies,$164.76,1.0,2018-01-01,One Star,"Came with parts missing (door cover, latch and...",0


In [62]:
df.shape

(551159, 14)

In [63]:
cols = df.columns
cols

Index(['userName', 'verified', 'itemName', 'description', 'image', 'brand',
       'feature', 'category', 'price', 'rating', 'reviewTime', 'summary',
       'reviewText', 'vote'],
      dtype='object')

In [64]:
# removing columns: verivied, image, feature
inter_cols =  ['userName', 'itemName', 'description', 'brand', 'category', 'price', 'rating', 'reviewTime', 'summary',
       'reviewText', 'vote']

In [65]:
df_2 = df.copy()

In [66]:
df_2 = df_2.drop(['verified', 'image', 'feature'], axis=1)

In [67]:
df_2.head()

,userName,itemName,description,brand,category,price,rating,reviewTime,summary,reviewText,vote
0,Amazon Customer,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...","[""Made from deliciously decadent ingredients, ...",Toblerone,Prime_Pantry,$1.63,5.0,2018-01-01,Five Stars,super smooth and yummy with crunchy bits,0
1,Carla,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded i...",Prince Of Peace,Grocery_and_Gourmet_Food,$6.40,5.0,2018-01-01,Five Stars,Perfect for kombucha,0
2,Jaclyn,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",Pet Champion,Pet_Supplies,$7.99,5.0,2018-01-01,she has an odd shape chest and her pull over h...,Finally a harness that fits my puppy. I really...,0
3,Christinne M,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,KOH-I-NOOR,Arts_Crafts_and_Sewing,$14.18,5.0,2018-01-01,Loving!!,I LOVE THEM!! I bought them at Micheals our of...,0
4,Sandy,"Parker Urban Premium Ebony Metal Chiseled, Bal...","[""Stunning craftsmanship based on a classic st...",Parker,Office_Products,NaN,5.0,2018-01-01,I love this pen,"I love this pen! I love the shape of it, the f...",0


In [68]:
def input_str(abc):
    if ('$'  not  in str(abc)):
       return  -1
    elif('-' in str(abc)):
        z = str(abc).replace('$','').replace(',','').replace(' ','').split('-')
        return  float((float(z[0]) + float(z[1]) )/2 )
    else:
        return  float(str(abc).replace('$','').replace(' ','').replace(',',''))

In [69]:
df_2['price'] = df_2['price'].apply(input_str)

In [70]:
new_latest_set  =  df_2[ df_2['price'] > -1   ]

In [71]:
df_2 = new_latest_set.drop_duplicates()

In [72]:
df_2.shape

(428529, 11)

In [73]:
df_2['category'].value_counts()

category
Pet_Supplies                   148297
Grocery_and_Gourmet_Food        73867
Patio_Lawn_and_Garden           55422
Office_Products                 48383
Cell_Phones_and_Accessories     33241
Arts_Crafts_and_Sewing          31101
Prime_Pantry                    14408
Musical_Instruments             13686
Industrial_and_Scientific        5271
Video_Games                      2762
Luxury_Beauty                    1679
AMAZON_FASHION                    217
Software                           82
All_Beauty                         75
Digital_Music                      33
Appliances                          5
Name: count, dtype: int64

In [74]:
df_3 = df_2.copy()

In [75]:
df_3.shape

(428529, 11)

In [76]:
df_3.head()

,userName,itemName,description,brand,category,price,rating,reviewTime,summary,reviewText,vote
0,Amazon Customer,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...","[""Made from deliciously decadent ingredients, ...",Toblerone,Prime_Pantry,1.63,5.0,2018-01-01,Five Stars,super smooth and yummy with crunchy bits,0
1,Carla,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded i...",Prince Of Peace,Grocery_and_Gourmet_Food,6.40,5.0,2018-01-01,Five Stars,Perfect for kombucha,0
2,Jaclyn,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",Pet Champion,Pet_Supplies,7.99,5.0,2018-01-01,she has an odd shape chest and her pull over h...,Finally a harness that fits my puppy. I really...,0
3,Christinne M,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,KOH-I-NOOR,Arts_Crafts_and_Sewing,14.18,5.0,2018-01-01,Loving!!,I LOVE THEM!! I bought them at Micheals our of...,0
6,patrick,Blue Buffalo Life Protection Formula Natural P...,['Because puppyhood is such an important stage...,Blue Buffalo,Pet_Supplies,28.99,5.0,2018-01-01,... and has had no problems with this food - B...,My 1 year old husky loves it - healthy and has...,0


In [77]:
# df_3.to_excel('Final_Reduced_Amazon_Review.xlsx') 

In [78]:
df_3[df_3['category'] == 'Pet_Supplies']['reviewText'].value_counts()

reviewText
great                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             225
good                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [79]:
# Ensure stopwords are loaded
stop_words = list(stopwords.words("english"))

def top_words_by_sentiment(df, category, n=5):
    """
    Extract the top positive and negative words for a specific category in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - category (str): The category to filter by.
    - n (int): The number of top words to return for positive and negative sentiment.
    
    Returns:
    - dict: Top positive and negative words for the specified category.
    """
    
    # Filter for the specified category
    category_df = df[df['category'] == category]
    
    # Separate positive and negative reviews based on 'vote' values
    positive_reviews = category_df[category_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = category_df[category_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Vectorize and remove stop words
    vectorizer = CountVectorizer(stop_words=stop_words)
    
    # Positive words frequency
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_words = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_words = sorted(positive_words.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative words frequency
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_words = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_words = sorted(negative_words.items(), key=lambda x: x[1], reverse=True)[:n]
    
    return {
        'top_positive_words': top_positive_words,
        'top_negative_words': top_negative_words
    }

# Example Usage: Top words in the 'Pet_Supplies' category
top_words_pet_supplies = top_words_by_sentiment(df_3, category="Pet_Supplies", n=5)
print(top_words_pet_supplies)


{'top_positive_words': [('dog', 32629), ('great', 26304), ('love', 24226), ('one', 20748), ('dogs', 18084)], 'top_negative_words': [('dog', 6269), ('one', 3872), ('like', 3586), ('would', 3318), ('dogs', 2689)]}


In [80]:
# Download required NLTK data
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("punkt")

# Define general stop words
general_stop_words = set(stopwords.words("english"))

def filter_sentiment_adjectives(text):
    """Filter adjectives that convey sentiment from text."""
    words = word_tokenize(text)
    filtered_words = [word for word, pos in pos_tag(words) if pos.startswith('JJ')]  # JJ for adjectives
    return ' '.join(filtered_words)

def top_words_by_sentiment(df, category, custom_stop_words=None, n=5):
    """
    Extract the top positive and negative sentiment adjectives for a specific category in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - category (str): The category to filter by.
    - custom_stop_words (set): A set of additional stop words specific to the category.
    - n (int): The number of top words to return for positive and negative sentiment.
    
    Returns:
    - dict: Top positive and negative sentiment adjectives for the specified category.
    """
    
    # Combine general and custom stop words
    stop_words = general_stop_words.union(custom_stop_words or set())
    
    # Filter for the specified category
    category_df = df[df['category'] == category]
    
    # Separate positive and negative reviews and filter for sentiment adjectives
    positive_reviews = category_df[category_df['rating'].isin([4, 5])]['reviewText'].fillna("").apply(filter_sentiment_adjectives)
    negative_reviews = category_df[category_df['rating'].isin([1, 2])]['reviewText'].fillna("").apply(filter_sentiment_adjectives)
    
    # Vectorize with TF-IDF and category-specific stop words
    vectorizer = TfidfVectorizer(stop_words=list(stop_words), max_df=0.85, min_df=2)
    
    # Positive sentiment words
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_words = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_words = sorted(positive_words.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative sentiment words
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_words = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_words = sorted(negative_words.items(), key=lambda x: x[1], reverse=True)[:n]
    
    return {
        'top_positive_words': top_positive_words,
        'top_negative_words': top_negative_words
    }

# Example usage for a category-specific stop word list
pet_stop_words = {"dog", "dogs", "cat", "cats", "food", "treat"}  # Example for pet-related products
top_words_pet_supplies = top_words_by_sentiment(df_3, category="Pet_Supplies", custom_stop_words=pet_stop_words, n=5)
print(top_words_pet_supplies)


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -3] Temporary failure in name resolution>
[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
{'top_positive_words': [('great', 8253.660612779175), ('good', 7058.677603197927), ('little', 3118.878655684278), ('easy', 2770.9115359051966), ('small', 2533.0291411789794)], 'top_negative_words': [('small', 764.9020565894398), ('good', 558.7144531856251), ('great', 337.5743304056612), ('first', 333.89991227127194), ('last', 326.41300810730183)]}


In [81]:
top_words_pet_supplies = top_words_by_sentiment(df_3, category="Grocery_and_Gourmet_Food", n=5)
print(top_words_pet_supplies)

{'top_positive_words': [('good', 7588.934658933644), ('great', 5171.929509889076), ('delicious', 2008.6091655102337), ('best', 1852.1313248432457), ('favorite', 1233.7901959783997)], 'top_negative_words': [('good', 370.45066798397363), ('much', 177.76805825518886), ('bad', 177.05061828724578), ('sweet', 143.56913338174968), ('disappointed', 140.50729616204572)]}


In [82]:
# Ensure stopwords are loaded and convert custom stop words to a list
stop_words = set(stopwords.words("english"))
custom_stop_words = list(stop_words.union({"amazon", "com", "product", "video", "image", "hidden", "class", "name", "box"}))  # Add custom stop words here

def top_bigrams_by_sentiment(df, category, n=5):
    """
    Extract the top positive and negative bigrams for a specific category in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - category (str): The category to filter by.
    - n (int): The number of top bigrams to return for positive and negative sentiment.
    
    Returns:
    - dict: Top positive and negative bigrams for the specified category.
    """
    
    # Filter for the specified category
    category_df = df[df['category'] == category]
    
    # Separate positive and negative reviews based on 'vote' values
    positive_reviews = category_df[category_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = category_df[category_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Vectorize and remove stop words, using TF-IDF and bigrams (2-word phrases)
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(2, 2), max_df=0.85)
    
    # Positive bigrams frequency
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_bigrams = sorted(positive_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative bigrams frequency
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_bigrams = sorted(negative_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    return {
        'top_positive_bigrams': top_positive_bigrams,
        'top_negative_bigrams': top_negative_bigrams
    }

# Example Usage: Top 5 bigrams in the 'Pet_Supplies' category
top_bigrams_pet_supplies = top_bigrams_by_sentiment(df_3, category="Pet_Supplies", n=10)
print(top_bigrams_pet_supplies)


{'top_positive_bigrams': [('dog loves', 1407.951338343776), ('dogs love', 1364.003153762774), ('cats love', 901.7169554828833), ('works great', 898.4260866067975), ('cat loves', 565.3566776417244), ('works well', 500.3906644937572), ('great price', 457.61542266446264), ('well made', 377.1061673684012), ('good quality', 361.18842299045906), ('good price', 300.4400018679215)], 'top_negative_bigrams': [('waste money', 96.17973205671959), ('dog like', 51.82463463744514), ('cats like', 47.313439353715495), ('dogs like', 43.06482715089331), ('way small', 38.1876348254157), ('cheaply made', 35.999750915832585), ('would eat', 34.711533148106085), ('dog eat', 34.18741552000601), ('work well', 32.25488409504563), ('small dog', 32.17754938953153)]}


In [83]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    # Remove special characters
    text = re.sub(r'\W', ' ', str(text))
    # Lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Applying preprocessing on the reviewText column
df_3['processed_reviewText'] = df_3['reviewText'].fillna('').apply(preprocess_text)

# Assign labels based on 'vote'
def assign_sentiment_label(rating):
    if rating in [4, 5]:
        return 'positive'
    elif rating in [1, 2]:
        return 'negative'
    else:
        return 'neutral'

# Add sentiment labels
df_3['sentiment'] = df_3['rating'].apply(assign_sentiment_label)

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    df_3['processed_reviewText'], df_3['sentiment'], test_size=0.25, random_state=42
)

# Define model pipeline
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Function to predict sentiment or appropriateness of new reviews
def check_review_sentiment(review):
    processed_review = preprocess_text(review)
    prediction = model.predict([processed_review])[0]
    return prediction

# Example usage
new_review = "This product is a scam and does not work!"
print("Review:", new_review)
print("Predicted Sentiment:", check_review_sentiment(new_review))


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Accuracy: 0.8547693054427674
Classification Report:
               precision    recall  f1-score   support

    negative       0.65      0.47      0.55     11033
     neutral       0.36      0.10      0.15      7964
    positive       0.88      0.97      0.92     88136

    accuracy                           0.85    107133
   macro avg       0.63      0.51      0.54    107133
weighted avg       0.82      0.85      0.83    107133

Review: This product is a scam and does not work!
Predicted Sentiment: negative


In [84]:
df_4 = df_2.copy()

In [85]:
df_4.shape # and head() to sanity check

(428529, 11)

In [86]:
# Calculate the total amount spent by each user
user_total_spent = df_4.groupby('userName')['price'].sum()
user_total_spent

userName
                      10.99
 #345876236754444     33.03
 (Kuro)                9.99
 Becca                57.84
 Berta Mae            10.34
                      ...  
~heavv~               33.97
~michelle~           172.35
~purplemoon~         138.86
~~Joe~~                2.86
~~Trish~~            189.69
Name: price, Length: 121208, dtype: float64

In [87]:
# Fill NaN values in 'userName' with a placeholder to avoid KeyError (bellow)
df_4['userName'] = df_4['userName'].fillna("Unknown")

In [88]:
unknown_user_count = df_4['userName'].value_counts().get('Unknown', 0)
print(f"Number of 'Unknown' user names: {unknown_user_count}")

Number of 'Unknown' user names: 121


In [89]:
df_4['userName'].value_counts().head(3)

userName
Amazon Customer    27196
Kindle Customer     2081
Mike                 504
Name: count, dtype: int64

In [90]:
# Adding a new column 'Amazon_verified' indicating if a user has spent more than $50
df_4['Amazon_verified'] = df_4['userName'].map(lambda user: 'Verified' if user_total_spent[user] >= 50 else 'Not Verified')

In [91]:
# Filter for specific usernames and get the value counts for 'Amazon_verified'
amazon_customer_verified_counts = df_4[df_4['userName'] == "Amazon Customer"]['Amazon_verified'].value_counts()
kindle_customer_verified_counts = df_4[df_4['userName'] == "Kindle Customer"]['Amazon_verified'].value_counts()

print("Amazon Customer Verification Status Counts:\n", amazon_customer_verified_counts)
print("\nKindle Customer Verification Status Counts:\n", kindle_customer_verified_counts)


Amazon Customer Verification Status Counts:
 Amazon_verified
Verified    27196
Name: count, dtype: int64

Kindle Customer Verification Status Counts:
 Amazon_verified
Verified    2081
Name: count, dtype: int64


In [92]:
df_4.shape # perfect - new column added

(428529, 12)

In [93]:
df_4['Amazon_verified'].value_counts()

Amazon_verified
Verified        301613
Not Verified    126916
Name: count, dtype: int64

In [94]:
# suggesting a new condition to leverage the comments:
# Calculate total transactions per user
user_transaction_counts = df_4['userName'].value_counts()

# Adding a new column 'Amazon_verified_levarage' with updated verification logic
df_4['Amazon_verified_leverage'] = df_4['userName'].map(
    lambda user: 'Verified' if (user_total_spent.get(user, 0) >= 30 or user_transaction_counts.get(user, 0) >= 3) else 'Not Verified'
)

# Display the counts of each status in the new column
print(df_4['Amazon_verified_leverage'].value_counts())

Amazon_verified_leverage
Verified        362280
Not Verified     66249
Name: count, dtype: int64


In [95]:
df_4.head()

,userName,itemName,description,brand,category,price,rating,reviewTime,summary,reviewText,vote,Amazon_verified,Amazon_verified_leverage
0,Amazon Customer,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...","[""Made from deliciously decadent ingredients, ...",Toblerone,Prime_Pantry,1.63,5.0,2018-01-01,Five Stars,super smooth and yummy with crunchy bits,0,Verified,Verified
1,Carla,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded i...",Prince Of Peace,Grocery_and_Gourmet_Food,6.40,5.0,2018-01-01,Five Stars,Perfect for kombucha,0,Verified,Verified
2,Jaclyn,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",Pet Champion,Pet_Supplies,7.99,5.0,2018-01-01,she has an odd shape chest and her pull over h...,Finally a harness that fits my puppy. I really...,0,Verified,Verified
3,Christinne M,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,KOH-I-NOOR,Arts_Crafts_and_Sewing,14.18,5.0,2018-01-01,Loving!!,I LOVE THEM!! I bought them at Micheals our of...,0,Not Verified,Not Verified
6,patrick,Blue Buffalo Life Protection Formula Natural P...,['Because puppyhood is such an important stage...,Blue Buffalo,Pet_Supplies,28.99,5.0,2018-01-01,... and has had no problems with this food - B...,My 1 year old husky loves it - healthy and has...,0,Verified,Verified


In [96]:
# Calculating the total sales for each product
product_sales = df_4.groupby('itemName')['price'].sum().sort_values(ascending=False)
product_sales

itemName
MidWest Homes for Pets Dog Crate | iCrate Single Door &amp; Double Door Folding Metal Dog Crates    22788.92
NIKE Women's Flex Supreme TR 4 Cross Trainer                                                        17498.00
Canon EOS Rebel T5 18MP EF-S Digital SLR Camera Bundle with Tripod and Accessories (16 Items)       16798.60
Purina Fortiflora Nutritional Supplement for Cats                                                   15114.60
Puppia Dog Harnesses                                                                                14830.66
                                                                                                      ...   
Apache OpenOffice 4.0.1 [Open Source Download]                                                          0.00
Amazon Music [Mac] [Download]                                                                           0.00
Kindle for PC [Download]                                                                                0.00
Adobe Flas

In [97]:
# Filter out rows where 'reviewText' is missing or empty, then count each itemName
reviewed_product_counts = df_4[df_4['reviewText'].notna() & (df_4['reviewText'].str.strip() != '')]['itemName'].value_counts()
reviewed_product_counts

itemName
Puppia Dog Harnesses                                                                                                             988
KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count                                             737
KIND Bars, Caramel Almond and Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count                                                    702
Kind Bars, Madagascar Vanilla Almond, Gluten Free, Low Sugar, 1.4oz                                                              664
KIND Bars, Dark Chocolate Chili Almond, Gluten Free, 1.4 Ounce Bars, 12 Count                                                    658
                                                                                                                                ... 
Safari Stainless Steel Double-Bladed Nail Trimmer for Cats, Kittens and Small Animals                                              1
Pringles 3 Flavor Snack Stacks, 12.69 Ounce (Pack of 4)     

In [98]:
# Calculating the total number of transactions for each product
product_transactions = df_4['itemName'].value_counts().sort_values(ascending=False)
product_transactions

itemName
Puppia Dog Harnesses                                                                                                                                                                               988
KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count                                                                                                               737
KIND Bars, Caramel Almond and Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count                                                                                                                      702
Kind Bars, Madagascar Vanilla Almond, Gluten Free, Low Sugar, 1.4oz                                                                                                                                664
KIND Bars, Dark Chocolate Chili Almond, Gluten Free, 1.4 Ounce Bars, 12 Count                                                                                                                      

In [99]:
# top 5
product_transactions.head(10)

itemName
Puppia Dog Harnesses                                                                    988
KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count    737
KIND Bars, Caramel Almond and Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count           702
Kind Bars, Madagascar Vanilla Almond, Gluten Free, Low Sugar, 1.4oz                     664
KIND Bars, Dark Chocolate Chili Almond, Gluten Free, 1.4 Ounce Bars, 12 Count           658
Voyager All Weather No Pull Step-in Mesh Dog Harness Padded Vest                        559
AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate                      456
AmazonBasics Padded Pet Bolster Bed                                                     440
AmazonBasics Pet Training and Puppy Pads                                                426
Milk-Bone Flavor Snacks Dog Treats                                                      408
Name: count, dtype: int64

In [100]:
# Displaying the first 5 comments (reviewText) for the item "Puppia Dog Harnesses" if available
puppia_harness_reviews = df_4[df_4['itemName'] == "Puppia Dog Harnesses"]['reviewText'].dropna().head(50)
puppia_harness_reviews

3250     If my little mutt wasn't as pudgy as I thought...
3251                                       Great Product!!
3252     Love this harness for our 7lb puppy! Originall...
3443     Love this harness for our 7lb puppy! Originall...
3444     If my little mutt wasn't as pudgy as I thought...
3458                                       Great Product!!
3506                                       Great Product!!
3507     If my little mutt wasn't as pudgy as I thought...
3509     Love this harness for our 7lb puppy! Originall...
10073                                              Perfect
10074                    Perfect for my puppy! & cute too!
10075    Needs different clips for keeping it together ...
10331    Needs different clips for keeping it together ...
10333                    Perfect for my puppy! & cute too!
10335                                              Perfect
10893                                              Perfect
10894    Needs different clips for keeping it together .

In [101]:
puppia_harness_reviews_positive = df_4[(df_4['itemName'] == "Puppia Dog Harnesses") & (df_4['rating'] == 2)]['reviewText'].dropna().head(10)
puppia_harness_reviews_positive

10075     Needs different clips for keeping it together ...
10331     Needs different clips for keeping it together ...
10894     Needs different clips for keeping it together ...
12316     I bought three Puppia soft dog harnesses for o...
12529     I bought three Puppia soft dog harnesses for o...
13616     I bought three Puppia soft dog harnesses for o...
88905     Followed the measurement chart and the harness...
89869     Followed the measurement chart and the harness...
90112     Followed the measurement chart and the harness...
103730    I purchased this same harness a few years ago ...
Name: reviewText, dtype: object

In [102]:
# for our DEMO we are going to use at least three different items and category: 
# 1 - Puppia Dog Harnesses                                                                    988
# 2 - KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count    737
# 3 - AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate                      456

# Now let's get the top 5 positives and negtive words!

In [103]:
# Download required NLTK data if needed
nltk.download("stopwords")
stop_words = list(stopwords.words("english"))

def top_words_by_item(df, item_name, n=5):
    """
    Extract the top positive and negative words for a specific item in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - item_name (str): The item name to filter by.
    - n (int): The number of top words to return for positive and negative sentiment.
    
    Returns:
    - dict: Top positive and negative words for the specified item.
    """
    
    # Filter for the specified item
    item_df = df[df['itemName'] == item_name]
    
    # Separate positive and negative reviews based on 'vote' values
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Define vectorizer once outside the conditionals
    vectorizer = CountVectorizer(stop_words=stop_words)
    
    # Ensure that positive and negative reviews contain content
    if positive_reviews.str.strip().eq("").all():
        top_positive_words = []
    else:
        positive_word_matrix = vectorizer.fit_transform(positive_reviews)
        positive_word_counts = positive_word_matrix.sum(axis=0).A1
        positive_words = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
        top_positive_words = sorted(positive_words.items(), key=lambda x: x[1], reverse=True)[:n]

    if negative_reviews.str.strip().eq("").all():
        top_negative_words = []
    else:
        negative_word_matrix = vectorizer.fit_transform(negative_reviews)
        negative_word_counts = negative_word_matrix.sum(axis=0).A1
        negative_words = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
        top_negative_words = sorted(negative_words.items(), key=lambda x: x[1], reverse=True)[:n]
    
    return {
        'top_positive_words': top_positive_words,
        'top_negative_words': top_negative_words
    }

# Example Usage: Top words for specific items
top_words_harness = top_words_by_item(df_4, item_name="Puppia Dog Harnesses", n=5)
print("Top words for Puppia Dog Harnesses:", top_words_harness)

top_words_kind_bars = top_words_by_item(df_4, item_name="KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count", n=5)
print("Top words for KIND Bars:", top_words_kind_bars)

top_words_dog_crate = top_words_by_item(df_4, item_name="AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate", n=5)
print("Top words for AmazonBasics Dog Crate:", top_words_dog_crate)


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Top words for Puppia Dog Harnesses: {'top_positive_words': [('harness', 310), ('dog', 240), ('great', 213), ('fits', 190), ('fit', 174)], 'top_negative_words': [('dog', 123), ('harness', 96), ('small', 90), ('fit', 63), ('one', 42)]}
Top words for KIND Bars: {'top_positive_words': [('bars', 166), ('great', 153), ('good', 132), ('love', 122), ('kind', 109)], 'top_negative_words': [('bars', 38), ('hard', 27), ('stale', 25), ('kind', 24), ('like', 16)]}
Top words for AmazonBasics Dog Crate: {'top_positive_words': [('crate', 211), ('great', 164), ('dog', 125), ('easy', 122), ('size', 83)], 'top_negative_words': [('dog', 8), ('one', 8), ('crate', 7), ('get', 6), ('good', 5)]}


In [104]:
# now let's take a look of the best pos and neg comms:

def top_reviews_by_item(df, item_name, n=5):
    """
    Extract the top positive and negative reviews for a specific item in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - item_name (str): The item name to filter by.
    - n (int): The number of top reviews to return for positive and negative sentiment.
    
    Returns:
    - dict: Top positive and negative reviews for the specified item.
    """
    
    # Filter for the specified item
    item_df = df[df['itemName'] == item_name]
    
    # Separate positive and negative reviews based on 'vote' values
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].dropna().head(n).tolist()
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].dropna().head(n).tolist()
    
    return {
        'top_positive_reviews': positive_reviews,
        'top_negative_reviews': negative_reviews
    }

# Example Usage: Top reviews for specific items
top_reviews_harness = top_reviews_by_item(df_4, item_name="Puppia Dog Harnesses", n=5)
print("Top reviews for Puppia Dog Harnesses:", top_reviews_harness)

top_reviews_kind_bars = top_reviews_by_item(df_4, item_name="KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count", n=5)
print("Top reviews for KIND Bars:", top_reviews_kind_bars)

top_reviews_dog_crate = top_reviews_by_item(df_4, item_name="AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate", n=5)
print("Top reviews for AmazonBasics Dog Crate:", top_reviews_dog_crate)


Top reviews for Puppia Dog Harnesses: {'top_positive_reviews': ['If my little mutt wasn\'t as pudgy as I thought she was this would fit perfectly...She seems to be a "tweener"..she\'s between a medium and large.  Other than that this is a very well constructed harness.', 'Great Product!!', "Love this harness for our 7lb puppy! Originally got the XS- because she's only a few months old, but her head was too big to slip through easily. the Small is a better fir for over her head, we had to tighten the belly strap all the way since it's a little big- but she'll grow into it nicley- really well made!", "Love this harness for our 7lb puppy! Originally got the XS- because she's only a few months old, but her head was too big to slip through easily. the Small is a better fir for over her head, we had to tighten the belly strap all the way since it's a little big- but she'll grow into it nicley- really well made!", 'If my little mutt wasn\'t as pudgy as I thought she was this would fit perfect

In [105]:
# Download required NLTK data if needed
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
custom_stop_words = list(stop_words.union({"amazon", "com", "product", "video", "image", "hidden", "class", "name", "box"}))

def top_bigrams_by_item(df, item_name, n=5):
    """
    Extract the top positive and negative bigrams for a specific item in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - item_name (str): The item name to filter by.
    - n (int): The number of top bigrams to return for positive and negative sentiment.
    
    Returns:
    - dict: Top positive and negative bigrams for the specified item.
    """
    
    # Filter for the specified item
    item_df = df[df['itemName'] == item_name]
    
    # Separate positive and negative reviews based on 'vote' values
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Vectorize and remove stop words, using TF-IDF and bigrams (2-word phrases)
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(2, 2), max_df=0.85)
    
    # Positive bigrams frequency
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_bigrams = sorted(positive_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative bigrams frequency
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_bigrams = sorted(negative_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    return {
        'top_positive_bigrams': top_positive_bigrams,
        'top_negative_bigrams': top_negative_bigrams
    }

# List of item names to analyze
item_names = [
    "Puppia Dog Harnesses",
    "KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count",
    "AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate"
]

# Loop through each item and get top bigrams
for item in item_names:
    bigrams = top_bigrams_by_item(df_4, item_name=item, n=10)
    print(f"Top bigrams for {item}:")
    print("Positive Bigrams:", bigrams['top_positive_bigrams'])
    print("Negative Bigrams:", bigrams['top_negative_bigrams'])
    print("\n")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Top bigrams for Puppia Dog Harnesses:
Positive Bigrams: [('easy put', 9.229998361334907), ('perfect fit', 8.305259415896758), ('well made', 7.119726239445134), ('great harness', 6.1433980477882315), ('fits great', 6.117566834734155), ('great quality', 5.8641802012273025), ('works great', 5.7964646994700075), ('great price', 5.6795568779879), ('comfortable dog', 5.422499542716795), ('fits well', 5.129372155070933)]
Negative Bigrams: [('large dog', 4.241086548762492), ('small medium', 4.168950991813728), ('way big', 3.405831484947769), ('innacurate size', 3.0), ('much small', 2.952840786365443), ('fit dog', 2.9232809307123815), ('put dog', 2.4174207100216414), ('medium large', 2.3291458835522985), ('sooo small', 2.3237780411320745), ('large fit', 2.3001418079761424)]


Top bigrams for KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count:
Pos

In [106]:
# now have unique outcomes: We're using this one

# Download required NLTK data if needed
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
custom_stop_words = list(stop_words.union({"amazon", "com", "product", "video", "image", "hidden", "class", "name", "box"}))

def top_unique_bigrams_by_item(df, item_name, n=5):
    """
    Extract the top unique positive and negative bigrams for a specific item in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - item_name (str): The item name to filter by.
    - n (int): The number of top bigrams to return for positive and negative sentiment.
    
    Returns:
    - dict: Unique top positive and negative bigrams for the specified item.
    """
    
    # Filter for the specified item
    item_df = df[df['itemName'] == item_name]
    
    # Separate positive and negative reviews based on 'rating' values
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Vectorize and remove stop words, using TF-IDF and bigrams (2-word phrases)
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(2, 2), max_df=0.85)
    
    # Positive bigrams frequency
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_bigrams = sorted(positive_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative bigrams frequency
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_bigrams = sorted(negative_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Remove overlapping bigrams to ensure uniqueness
    positive_set = set(word for word, count in top_positive_bigrams)
    negative_set = set(word for word, count in top_negative_bigrams)
    
    unique_positive_bigrams = [(word, count) for word, count in top_positive_bigrams if word not in negative_set]
    unique_negative_bigrams = [(word, count) for word, count in top_negative_bigrams if word not in positive_set]
    
    return {
        'top_positive_bigrams': unique_positive_bigrams[:n],
        'top_negative_bigrams': unique_negative_bigrams[:n]
    }

# List of item names to analyze
item_names = [
    "Puppia Dog Harnesses",
    "KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count",
    "AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate"
]

# Loop through each item and get top unique bigrams
for item in item_names:
    bigrams = top_unique_bigrams_by_item(df_4, item_name=item, n=10)
    print(f"Top unique bigrams for {item}:")
    print("Positive Bigrams:", bigrams['top_positive_bigrams'])
    print("Negative Bigrams:", bigrams['top_negative_bigrams'])
    print("\n")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Top unique bigrams for Puppia Dog Harnesses:
Positive Bigrams: [('easy put', 9.229998361334907), ('perfect fit', 8.305259415896758), ('well made', 7.119726239445134), ('great harness', 6.1433980477882315), ('fits great', 6.117566834734155), ('great quality', 5.8641802012273025), ('works great', 5.7964646994700075), ('great price', 5.6795568779879), ('comfortable dog', 5.422499542716795), ('fits well', 5.129372155070933)]
Negative Bigrams: [('large dog', 4.241086548762492), ('small medium', 4.168950991813728), ('way big', 3.405831484947769), ('innacurate size', 3.0), ('much small', 2.952840786365443), ('fit dog', 2.9232809307123815), ('put dog', 2.4174207100216414), ('medium large', 2.3291458835522985), ('sooo small', 2.3237780411320745), ('large fit', 2.3001418079761424)]


Top unique bigrams for KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars,

In [107]:
#  Now only using the data where Amazon_verified = 'Verified'

# Download required NLTK data if needed
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
custom_stop_words = list(stop_words.union({"amazon", "com", "product", "video", "image", "hidden", "class", "name", "box"}))

def top_unique_bigrams_by_item_verified(df, item_name, n=5):
    """
    Extract the top unique positive and negative bigrams for a specific item in a dataset,
    considering only rows where Amazon_verified is "Verified".
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - item_name (str): The item name to filter by.
    - n (int): The number of top bigrams to return for positive and negative sentiment.
    
    Returns:
    - dict: Unique top positive and negative bigrams for the specified item.
    """
    
    # Filter for the specified item and verified reviews
    item_df = df[(df['itemName'] == item_name) & (df['Amazon_verified'] == "Verified")]
    
    # Separate positive and negative reviews based on 'rating' values
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Vectorize and remove stop words, using TF-IDF and bigrams (2-word phrases)
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(2, 2), max_df=0.85)
    
    # Positive bigrams frequency
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_bigrams = sorted(positive_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative bigrams frequency
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_bigrams = sorted(negative_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Remove overlapping bigrams to ensure uniqueness
    positive_set = set(word for word, count in top_positive_bigrams)
    negative_set = set(word for word, count in top_negative_bigrams)
    
    unique_positive_bigrams = [(word, count) for word, count in top_positive_bigrams if word not in negative_set]
    unique_negative_bigrams = [(word, count) for word, count in top_negative_bigrams if word not in positive_set]
    
    return {
        'top_positive_bigrams': unique_positive_bigrams[:n],
        'top_negative_bigrams': unique_negative_bigrams[:n]
    }

# List of item names to analyze
item_names = [
    "Puppia Dog Harnesses",
    "KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count",
    "AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate"
]

# Loop through each item and get top unique bigrams
for item in item_names:
    bigrams = top_unique_bigrams_by_item_verified(df_4, item_name=item, n=10)
    print(f"Top unique bigrams for {item}:")
    print("Positive Bigrams:", bigrams['top_positive_bigrams'])
    print("Negative Bigrams:", bigrams['top_negative_bigrams'])
    print("\n")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Top unique bigrams for Puppia Dog Harnesses:
Positive Bigrams: [('easy put', 6.45615521004141), ('perfect fit', 5.2582768948384935), ('fits well', 4.340687685093655), ('good price', 3.965540497592226), ('well made', 3.9118036331256314), ('small dog', 3.759765058935519), ('fit perfectly', 3.711476233705194), ('good harness', 3.616027958379303), ('looks great', 3.5934487043727774), ('great quality', 3.0765291166618107)]
Negative Bigrams: [('innacurate size', 3.0), ('large dog', 2.888746597803891), ('way big', 2.4339700739445354), ('small medium', 2.3033327249943687), ('large fit', 2.2153715255723414), ('medium large', 2.176987397049947), ('accurate return', 2.121320343559643), ('bog little', 2.121320343559643), ('little one', 2.121320343559643), ('made pup', 2.121320343559643)]


Top unique bigrams for KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce B

In [108]:
# Now only using the data where Amazon_verified_leverage = 'Verified'

# Download required NLTK data if needed
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
custom_stop_words = list(stop_words.union({"amazon", "com", "product", "video", "image", "hidden", "class", "name", "box"}))

def top_unique_bigrams_by_item_verified_leverage(df, item_name, n=5):
    """
    Extract the top unique positive and negative bigrams for a specific item in a dataset,
    considering only rows where Amazon_verified_leverage is "Verified".
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - item_name (str): The item name to filter by.
    - n (int): The number of top bigrams to return for positive and negative sentiment.
    
    Returns:
    - dict: Unique top positive and negative bigrams for the specified item.
    """
    
    # Filter for the specified item and leverage-verified reviews
    item_df = df[(df['itemName'] == item_name) & (df['Amazon_verified_leverage'] == "Verified")]
    
    # Separate positive and negative reviews based on 'rating' values
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Vectorize and remove stop words, using TF-IDF and bigrams (2-word phrases)
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(2, 2), max_df=0.85)
    
    # Positive bigrams frequency
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_bigrams = sorted(positive_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative bigrams frequency
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_bigrams = sorted(negative_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Remove overlapping bigrams to ensure uniqueness
    positive_set = set(word for word, count in top_positive_bigrams)
    negative_set = set(word for word, count in top_negative_bigrams)
    
    unique_positive_bigrams = [(word, count) for word, count in top_positive_bigrams if word not in negative_set]
    unique_negative_bigrams = [(word, count) for word, count in top_negative_bigrams if word not in positive_set]
    
    return {
        'top_positive_bigrams': unique_positive_bigrams[:n],
        'top_negative_bigrams': unique_negative_bigrams[:n]
    }

# List of item names to analyze
item_names = [
    "Puppia Dog Harnesses",
    "KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count",
    "AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate"
]

# Loop through each item and get top unique bigrams
for item in item_names:
    bigrams = top_unique_bigrams_by_item_verified_leverage(df_4, item_name=item, n=10)
    print(f"Top unique bigrams for {item}:")
    print("Positive Bigrams:", bigrams['top_positive_bigrams'])
    print("Negative Bigrams:", bigrams['top_negative_bigrams'])
    print("\n")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Top unique bigrams for Puppia Dog Harnesses:
Positive Bigrams: [('easy put', 9.229998361334907), ('perfect fit', 8.305259415896758), ('well made', 7.119726239445134), ('great harness', 6.1433980477882315), ('fits great', 6.117566834734155), ('great quality', 5.8641802012273025), ('works great', 5.7964646994700075), ('great price', 5.6795568779879), ('comfortable dog', 5.422499542716795), ('fits well', 5.129372155070933)]
Negative Bigrams: [('large dog', 4.241086548762492), ('small medium', 4.168950991813728), ('way big', 3.405831484947769), ('innacurate size', 3.0), ('much small', 2.952840786365443), ('fit dog', 2.9232809307123815), ('put dog', 2.4174207100216414), ('medium large', 2.3291458835522985), ('sooo small', 2.3237780411320745), ('large fit', 2.3001418079761424)]


Top unique bigrams for KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars,

In [109]:
# Download required NLTK data
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
custom_stop_words = list(stop_words.union({"amazon", "com", "product", "video", "image", "hidden", "class", "name", "box"}))

# Define the function to extract unique bigrams
def top_unique_bigrams_by_item_verified_leverage(df, item_name, n=5):
    item_df = df[(df['itemName'] == item_name) & (df['Amazon_verified_leverage'] == "Verified")]
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(2, 2), max_df=0.85)

    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_bigrams = sorted(positive_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]

    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_bigrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_bigrams = sorted(negative_bigrams.items(), key=lambda x: x[1], reverse=True)[:n]

    positive_set = set(word for word, count in top_positive_bigrams)
    negative_set = set(word for word, count in top_negative_bigrams)

    unique_positive_bigrams = [word for word, count in top_positive_bigrams if word not in negative_set]
    unique_negative_bigrams = [word for word, count in top_negative_bigrams if word not in positive_set]
    
    return {
        'top_positive_bigrams': unique_positive_bigrams if unique_positive_bigrams else "TBC",
        'top_negative_bigrams': unique_negative_bigrams if unique_negative_bigrams else "TBC"
    }

# Initialize new columns with default value "TBC"
df_4['Positive Phrases'] = "TBC"
df_4['Negative Phrases'] = "TBC"

# Define the items to analyze
item_names = [
    "Puppia Dog Harnesses",
    "KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count",
    "AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate"
]

# Update each row based on the item name and verified leverage bigrams
for item in item_names:
    bigrams = top_unique_bigrams_by_item_verified_leverage(df_4, item_name=item, n=10)
    df_4.loc[df_4['itemName'] == item, 'Positive Phrases'] = df_4.loc[df_4['itemName'] == item].apply(
        lambda x: bigrams['top_positive_bigrams'], axis=1
    )
    df_4.loc[df_4['itemName'] == item, 'Negative Phrases'] = df_4.loc[df_4['itemName'] == item].apply(
        lambda x: bigrams['top_negative_bigrams'], axis=1
    )

# Display the unique items along with their Positive and Negative Phrases
# Here, we display only one sample row per item to avoid duplications visually

for item in item_names:
    sample_row = df_4[df_4['itemName'] == item][['itemName', 'Positive Phrases', 'Negative Phrases']].iloc[0]
    print(f"Item Name: {sample_row['itemName']}")
    print("Positive Phrases:", sample_row['Positive Phrases'])
    print("Negative Phrases:", sample_row['Negative Phrases'])
    print("\n")

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Item Name: Puppia Dog Harnesses
Positive Phrases: ['easy put', 'perfect fit', 'well made', 'great harness', 'fits great', 'great quality', 'works great', 'great price', 'comfortable dog', 'fits well']
Negative Phrases: ['large dog', 'small medium', 'way big', 'innacurate size', 'much small', 'fit dog', 'put dog', 'medium large', 'sooo small', 'large fit']


Item Name: KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count
Positive Phrases: ['great snack', 'love bars', 'great taste', 'favorite kind', 'kind bar', 'taste great', 'dark chocolate', 'gluten free', 'best flavor']
Negative Phrases: ['bars stale', 'rock hard', 'tasted stale', 'hard rock', 'stale hard', 'hard brick', 'stale stale', 'taste good', 'hard rocks']


Item Name: AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate
Positive Phrases: ['great price', 'great crate'

In [110]:
df_4.head(5)

,userName,itemName,description,brand,category,price,rating,reviewTime,summary,reviewText,vote,Amazon_verified,Amazon_verified_leverage,Positive Phrases,Negative Phrases
0,Amazon Customer,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...","[""Made from deliciously decadent ingredients, ...",Toblerone,Prime_Pantry,1.63,5.0,2018-01-01,Five Stars,super smooth and yummy with crunchy bits,0,Verified,Verified,TBC,TBC
1,Carla,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags","['Prince of Peace Enterprises, Inc., founded i...",Prince Of Peace,Grocery_and_Gourmet_Food,6.40,5.0,2018-01-01,Five Stars,Perfect for kombucha,0,Verified,Verified,TBC,TBC
2,Jaclyn,"Pet Champion Adjustable No-Pull Harness, Colla...","[""The Pet Champion Large/ Extra Large 22-36 in...",Pet Champion,Pet_Supplies,7.99,5.0,2018-01-01,she has an odd shape chest and her pull over h...,Finally a harness that fits my puppy. I really...,0,Verified,Verified,TBC,TBC
3,Christinne M,Koh-I-Noor Progresso Woodless Colored 24-Penci...,['Koh-I-Noor Progresso Woodless Colored 24-Pen...,KOH-I-NOOR,Arts_Crafts_and_Sewing,14.18,5.0,2018-01-01,Loving!!,I LOVE THEM!! I bought them at Micheals our of...,0,Not Verified,Not Verified,TBC,TBC
6,patrick,Blue Buffalo Life Protection Formula Natural P...,['Because puppyhood is such an important stage...,Blue Buffalo,Pet_Supplies,28.99,5.0,2018-01-01,... and has had no problems with this food - B...,My 1 year old husky loves it - healthy and has...,0,Verified,Verified,TBC,TBC


In [111]:
inter_cols2 =  ['itemName', 'Positive Phrases', 'Negative Phrases']
df_4[inter_cols2].head(10)

,itemName,Positive Phrases,Negative Phrases
0,"Toblerone Swiss Milk Chocolate Bar, Crunchy Sa...",TBC,TBC
1,"Prince of Peace Organic Tea, Oolong, 100 Tea Bags",TBC,TBC
2,"Pet Champion Adjustable No-Pull Harness, Colla...",TBC,TBC
3,Koh-I-Noor Progresso Woodless Colored 24-Penci...,TBC,TBC
6,Blue Buffalo Life Protection Formula Natural P...,TBC,TBC
7,Squishy Face Studio Flirt Pole V2 with Braided...,TBC,TBC
8,Penn Plax Net Breeder for Aquarium,TBC,TBC
9,PetSafe Freedom Aluminum Patio Panel Sliding G...,TBC,TBC
10,Vet Recommended Pet ID Tag Dog and Cat Persona...,TBC,TBC
11,"Avery Removable Print or Write Labels, 1.5 x 3...",TBC,TBC


In [112]:
# for tableau
df_4[inter_cols2].to_excel('nlp.xlsx') 

In [113]:
# Download required NLTK data if needed
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
custom_stop_words = list(stop_words.union({"amazon", "com", "product", "video", "image", "hidden", "class", "name", "box"}))

def top_unique_threegrams_by_item(df, item_name, n=5):
    """
    Extract the top unique positive and negative three-grams for a specific item in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - item_name (str): The item name to filter by.
    - n (int): The number of top three-grams to return for positive and negative sentiment.
    
    Returns:
    - dict: Unique top positive and negative three-grams for the specified item.
    """
    
    # Filter for the specified item
    item_df = df[df['itemName'] == item_name]
    
    # Separate positive and negative reviews based on 'rating' values
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Vectorize and remove stop words, using TF-IDF and three-grams (3-word phrases)
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(3, 3), max_df=0.85)
    
    # Positive three-grams frequency
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_threegrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_threegrams = sorted(positive_threegrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative three-grams frequency
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_threegrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_threegrams = sorted(negative_threegrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Remove overlapping three-grams to ensure uniqueness
    positive_set = set(word for word, count in top_positive_threegrams)
    negative_set = set(word for word, count in top_negative_threegrams)
    
    unique_positive_threegrams = [(word, count) for word, count in top_positive_threegrams if word not in negative_set]
    unique_negative_threegrams = [(word, count) for word, count in top_negative_threegrams if word not in positive_set]
    
    return {
        'top_positive_threegrams': unique_positive_threegrams[:n],
        'top_negative_threegrams': unique_negative_threegrams[:n]
    }

# List of item names to analyze
item_names = [
    "Puppia Dog Harnesses",
    "KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count",
    "AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate"
]

# Loop through each item and get top unique three-grams
for item in item_names:
    threegrams = top_unique_threegrams_by_item(df_4, item_name=item, n=10)
    print(f"Top unique three-grams for {item}:")
    print("Positive Threegrams:", threegrams['top_positive_threegrams'])
    print("Negative Threegrams:", threegrams['top_negative_threegrams'])
    print("\n")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Top unique three-grams for Puppia Dog Harnesses:
Positive Threegrams: [('love easy put', 3.3523849902914358), ('best harnes ever', 3.0), ('expected good harness', 3.0), ('fits great loved', 3.0), ('fits westie great', 3.0), ('good small dog', 3.0), ('love puppia products', 3.0), ('perfect fit chihuahua', 3.0), ('perfect fit schnorkie', 3.0), ('perfect puppy cute', 3.0)]
Negative Threegrams: [('bog little one', 3.0), ('made pup itch', 3.0), ('size accurate return', 3.0), ('sooo small medium', 3.0), ('hates going head', 2.1213203435596424), ('medium smaller xsmall', 2.1213203435596424), ('ordered medium smaller', 2.1213203435596424), ('ordered xxl received', 2.1213203435596424), ('puppy hates going', 2.1213203435596424), ('sizing way way', 2.1213203435596424)]


Top unique three-grams for KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count:

In [114]:
# Download required NLTK data if needed
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
custom_stop_words = list(stop_words.union({"amazon", "com", "product", "video", "image", "hidden", "class", "name", "box"}))

def top_unique_fourgrams_by_item(df, item_name, n=5):
    """
    Extract the top unique positive and negative four-grams for a specific item in a dataset.
    
    Parameters:
    - df (DataFrame): The dataset containing the reviews.
    - item_name (str): The item name to filter by.
    - n (int): The number of top four-grams to return for positive and negative sentiment.
    
    Returns:
    - dict: Unique top positive and negative four-grams for the specified item.
    """
    
    # Filter for the specified item
    item_df = df[df['itemName'] == item_name]
    
    # Separate positive and negative reviews based on 'rating' values
    positive_reviews = item_df[item_df['rating'].isin([4, 5])]['reviewText'].fillna("")
    negative_reviews = item_df[item_df['rating'].isin([1, 2])]['reviewText'].fillna("")
    
    # Vectorize and remove stop words, using TF-IDF and four-grams (4-word phrases)
    vectorizer = TfidfVectorizer(stop_words=custom_stop_words, ngram_range=(4, 4), max_df=0.85)
    
    # Positive four-grams frequency
    positive_word_matrix = vectorizer.fit_transform(positive_reviews)
    positive_word_counts = positive_word_matrix.sum(axis=0).A1
    positive_fourgrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), positive_word_counts)}
    top_positive_fourgrams = sorted(positive_fourgrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Negative four-grams frequency
    negative_word_matrix = vectorizer.fit_transform(negative_reviews)
    negative_word_counts = negative_word_matrix.sum(axis=0).A1
    negative_fourgrams = {word: count for word, count in zip(vectorizer.get_feature_names_out(), negative_word_counts)}
    top_negative_fourgrams = sorted(negative_fourgrams.items(), key=lambda x: x[1], reverse=True)[:n]
    
    # Remove overlapping four-grams to ensure uniqueness
    positive_set = set(word for word, count in top_positive_fourgrams)
    negative_set = set(word for word, count in top_negative_fourgrams)
    
    unique_positive_fourgrams = [(word, count) for word, count in top_positive_fourgrams if word not in negative_set]
    unique_negative_fourgrams = [(word, count) for word, count in top_negative_fourgrams if word not in positive_set]
    
    return {
        'top_positive_fourgrams': unique_positive_fourgrams[:n],
        'top_negative_fourgrams': unique_negative_fourgrams[:n]
    }

# List of item names to analyze
item_names = [
    "Puppia Dog Harnesses",
    "KIND Bars, Dark Chocolate Nuts &amp; Sea Salt, Gluten Free, 1.4 Ounce Bars, 12 Count",
    "AmazonBasics Single Door &amp; Double Door Folding Metal Dog Crate"
]

# Loop through each item and get top unique four-grams
for item in item_names:
    fourgrams = top_unique_fourgrams_by_item(df_4, item_name=item, n=10)
    print(f"Top unique four-grams for {item}:")
    print("Positive Fourgrams:", fourgrams['top_positive_fourgrams'])
    print("Negative Fourgrams:", fourgrams['top_negative_fourgrams'])
    print("\n")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Top unique four-grams for Puppia Dog Harnesses:
Positive Fourgrams: [('bandit loves new swag', 3.0), ('brand harness buy pups', 3.0), ('dog loves sturdy comfortable', 3.0), ('easy put attachment sturdy', 3.0), ('fit 4lb teacup yorkie', 3.0), ('good would recommend friend', 3.0), ('great halter great price', 3.0), ('harness great dog bit', 3.0), ('looks works great bichon', 3.0), ('love boy like take', 3.0)]
Negative Fourgrams: [('ordered medium smaller xsmall', 3.0), ('ordered xxl received small', 3.0), ('puppy hates going head', 3.0), ('sizing way way big', 3.0), ('described cat even wear', 2.1213203435596424), ('even though measured dog', 2.1213203435596424), ('fall apart first uses', 2.1213203435596424), ('fit even though measured', 2.1213203435596424), ('fit large dog tiny', 2.1213203435596424), ('harness runs much small', 2.1213203435596424)]


Top unique four-grams

In [115]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    # Remove special characters and lowercase text
    text = re.sub(r'\W', ' ', str(text)).lower()
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Assign sentiment labels based on 'rating'
def assign_sentiment_label(rating):
    if rating in [4, 5]:
        return 'positive'
    elif rating in [1, 2]:
        return 'negative'
    else:
        return 'neutral'

# Preprocess reviewText and assign sentiment labels
df_4['processed_reviewText'] = df_4['reviewText'].fillna('').apply(preprocess_text)
df_4['sentiment'] = df_4['rating'].apply(assign_sentiment_label)

def evaluate_sentiment_model(df, verification_column):
    """
    Train and evaluate a sentiment prediction model using a specific verification column.
    
    Parameters:
    - df (DataFrame): The dataset containing reviews and verification status.
    - verification_column (str): The verification column to filter on ('Amazon_verified' or 'Amazon_verified_leverage').

    Returns:
    - Prints accuracy and classification report for the model trained on the filtered data.
    """
    # Filter the data for rows where the specified verification column is 'Verified'
    verified_df = df[df[verification_column] == 'Verified']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        verified_df['processed_reviewText'], verified_df['sentiment'], test_size=0.25, random_state=42
    )

    # Define and train the model
    model = make_pipeline(CountVectorizer(), MultinomialNB())
    model.fit(X_train, y_train)

    # Make predictions and evaluate
    y_pred = model.predict(X_test)
    print(f"Results for {verification_column} = Verified:")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("\n")

# Evaluate the model for both verification conditions
evaluate_sentiment_model(df_4, verification_column="Amazon_verified")
evaluate_sentiment_model(df_4, verification_column="Amazon_verified_leverage")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
Results for Amazon_verified = Verified:
Accuracy: 0.8601798313086839
Classification Report:
               precision    recall  f1-score   support

    negative       0.65      0.43      0.52      7054
     neutral       0.37      0.09      0.14      5542
    positive       0.88      0.98      0.93     62808

    accuracy                           0.86     75404
   macro avg       0.64      0.50      0.53     75404
weighted avg       0.82      0.86      0.83     75404



Results for Amazon_verified_leverage = Verified:
Accuracy: 0.8595892679695263
Classification Report:
               precision    recall  f1-score   support

    negative       0.65      0.46      0.54      8734
     neutral       0.35      0.09      0.15      6546
    positive       0.89      0.97      0.93     75290

    accuracy                           0.86     90570
   macro avg       0.63      0.51

In [116]:
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    text = re.sub(r'\W', ' ', str(text)).lower()  # Remove special characters and lowercase
    return ' '.join([word for word in text.split() if word not in stop_words])

# Assign sentiment labels based on 'rating'
def assign_sentiment_label(rating):
    if rating in [4, 5]:
        return 'positive'
    elif rating in [1, 2]:
        return 'negative'
    else:
        return 'neutral'

# Preprocess reviewText and assign sentiment labels
df_4['processed_reviewText'] = df_4['reviewText'].fillna('').apply(preprocess_text)
df_4['sentiment'] = df_4['rating'].apply(assign_sentiment_label)

def evaluate_sentiment_model(df, verification_column):
    """
    Train and evaluate a sentiment prediction model using Logistic Regression on a specific verification column.
    
    Parameters:
    - df (DataFrame): The dataset containing reviews and verification status.
    - verification_column (str): The verification column to filter on ('Amazon_verified' or 'Amazon_verified_leverage').

    Returns:
    - Prints accuracy and classification report for the model trained on the filtered data.
    """
    # Filter the data for rows where the specified verification column is 'Verified'
    verified_df = df[df[verification_column] == 'Verified']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        verified_df['processed_reviewText'], verified_df['sentiment'], test_size=0.25, random_state=42
    )

    # Define and train the Logistic Regression model
    model = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=200, C=10))
    model.fit(X_train, y_train)

    # Make predictions and evaluate
    y_pred = model.predict(X_test)
    print(f"Results for {verification_column} = Verified (Logistic Regression):")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("\n")

# Evaluate the model for both verification conditions
evaluate_sentiment_model(df_4, verification_column="Amazon_verified")
evaluate_sentiment_model(df_4, verification_column="Amazon_verified_leverage")


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results for Amazon_verified = Verified (Logistic Regression):
Accuracy: 0.8566654288897141
Classification Report:
               precision    recall  f1-score   support

    negative       0.61      0.50      0.55      7054
     neutral       0.32      0.19      0.24      5542
    positive       0.90      0.96      0.93     62808

    accuracy                           0.86     75404
   macro avg       0.61      0.55      0.57     75404
weighted avg       0.83      0.86      0.84     75404





/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Results for Amazon_verified_leverage = Verified (Logistic Regression):
Accuracy: 0.8572706194104008
Classification Report:
               precision    recall  f1-score   support

    negative       0.61      0.54      0.57      8734
     neutral       0.30      0.18      0.23      6546
    positive       0.91      0.95      0.93     75290

    accuracy                           0.86     90570
   macro avg       0.61      0.56      0.58     90570
weighted avg       0.84      0.86      0.84     90570





In [117]:
# Define the pipeline with CountVectorizer and Logistic Regression
pipeline = make_pipeline(CountVectorizer(), LogisticRegression())

# Define the parameter grid
param_grid = {
    'logisticregression__C': [0.01, 0.1, 1, 10, 100],   # Different levels of regularization strength
    'logisticregression__max_iter': [100, 200, 500],     # Maximum iterations to ensure convergence
}

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit the grid search on the training data (replace X_train and y_train with your data)
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Print the best hyperparameters found
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best parameters on the test set
y_pred = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 15 candidates, totalling 75 fits


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Best Hyperparameters: {'logisticregression__C': 0.1, 'logisticregression__max_iter': 500}
Accuracy: 0.8625353532524992
Classification Report:
               precision    recall  f1-score   support

    negative       0.69      0.49      0.57     11033
     neutral       0.42      0.12      0.19      7964
    positive       0.89      0.98      0.93     88136

    accuracy                           0.86    107133
   macro avg       0.67      0.53      0.56    107133
weighted avg       0.83      0.86      0.84    107133

[CV] END logisticregression__C=0.01, logisticregression__max_iter=100; total time=  23.7s
[CV] END logisticregression__C=0.01, logisticregression__max_iter=200; total time=  24.5s
[CV] END logisticregression__C=0.01, logisticregression__max_iter=200; total time=  25.9s
[CV] END logisticregression__C=0.01, logisticregression__max_iter=500; total time=  26.1s
[CV] END logisticregression__C=0.1, logisticregression__max_iter=100; total time=  30.2s
[CV] END logisticregression